# Script for loading Nursing Homes source data from data.cms.gov into SQL Server

In [1]:
import pyodbc
import pandas as pd
from sodapy import Socrata

Read the source data from data.cms.gov

In [2]:
client = Socrata("data.cms.gov", None)
results = client.get_all("s2uc-8wxp")
df = pd.DataFrame.from_records(results)
df.head(10)

230503


,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,submitted_data,county,geolocation,...,has_facility_performed_staff_and_or_personnel_tests_since_last_report,tested_staff_and_or_personnel_with_new_signs_or_symptoms,tested_asymptomatic_staff_and_or_personnel_in_a_unit_or_section_after_a_new_case,tested_asymptomatic_staff_and_or_personnel_facility_wide_after_a_new_case,tested_asymptomatic_staff_and_or_personnel_without_known_exposure_as_surveillance,tested_another_subgroup_of_staff_and_or_personnel,number_of_ventilators_in,number_of_ventilators_in_1,any_current_supply_of,one_week_supply_of_ventilator
0,2020-05-24T00:00:00.000,035242,CHINLE NURSING HOME,PO BOX 910,CHINLE,AZ,86503,N,Apache,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-05-24T00:00:00.000,075195,PARKWAY PAVILION HEALTH AND REHABILITATION CENTER,1157 ENFIELD STREET,ENFIELD,CT,06082,N,Hartford,"{'type': 'Point', 'coordinates': [-72.593179, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-05-24T00:00:00.000,105384,CALUSA HARBOUR,2525 FIRST ST,FORT MYERS,FL,33901,N,Lee,"{'type': 'Point', 'coordinates': [-81.864238, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-05-31T00:00:00.000,105597,SURREY PLACE CARE CENTER,110 SE LEE AVE,LIVE OAK,FL,32064,N,Suwannee,"{'type': 'Point', 'coordinates': [-82.97076400...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-05-24T00:00:00.000,105697,EAST BAY REHABILITATION CENTER,4470 E BAY DR,CLEARWATER,FL,33764,N,Pinellas,"{'type': 'Point', 'coordinates': [-82.740891, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-05-24T00:00:00.000,105803,HIALEAH NURSING AND REHABILITATION CENTER,190 W 28TH STREET,HIALEAH,FL,33010,N,Miami-Dade,"{'type': 'Point', 'coordinates': [-80.286094, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2020-05-24T00:00:00.000,105812,ROYAL CARE OF AVON PARK,1213 W STRATFORD RD,AVON PARK,FL,33825,N,Highlands,"{'type': 'Point', 'coordinates': [-81.51721, 2...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2020-05-24T00:00:00.000,105832,"GARDENS OF PORT ST LUCIE, THE",1699 SE LYNGATE DRIVE,PORT SAINT LUCIE,FL,34952,N,St. Lucie,"{'type': 'Point', 'coordinates': [-80.298828, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2020-05-24T00:00:00.000,105951,BRANDON HEALTH AND REHABILITATION CENTER,1465 OAKFIELD DR,BRANDON,FL,33511,N,Hillsborough,"{'type': 'Point', 'coordinates': [-82.307614, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2020-05-24T00:00:00.000,106040,WESTMINSTER ST AUGUSTINE,230 TOWERVIEW DRIVE,SAINT AUGUSTINE,FL,32092,N,St. Johns,"{'type': 'Point', 'coordinates': [-81.480016, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Convert date type columns and calculate WeekNo (week number) column

In [3]:
df['week_ending']=pd.to_datetime(df['week_ending'])
df=df.assign(WeekNo=lambda x: (x['week_ending'].dt.isocalendar().week))
df.head(10)

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,submitted_data,county,geolocation,...,tested_staff_and_or_personnel_with_new_signs_or_symptoms,tested_asymptomatic_staff_and_or_personnel_in_a_unit_or_section_after_a_new_case,tested_asymptomatic_staff_and_or_personnel_facility_wide_after_a_new_case,tested_asymptomatic_staff_and_or_personnel_without_known_exposure_as_surveillance,tested_another_subgroup_of_staff_and_or_personnel,number_of_ventilators_in,number_of_ventilators_in_1,any_current_supply_of,one_week_supply_of_ventilator,WeekNo
0,2020-05-24,035242,CHINLE NURSING HOME,PO BOX 910,CHINLE,AZ,86503,N,Apache,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21
1,2020-05-24,075195,PARKWAY PAVILION HEALTH AND REHABILITATION CENTER,1157 ENFIELD STREET,ENFIELD,CT,06082,N,Hartford,"{'type': 'Point', 'coordinates': [-72.593179, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21
2,2020-05-24,105384,CALUSA HARBOUR,2525 FIRST ST,FORT MYERS,FL,33901,N,Lee,"{'type': 'Point', 'coordinates': [-81.864238, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21
3,2020-05-31,105597,SURREY PLACE CARE CENTER,110 SE LEE AVE,LIVE OAK,FL,32064,N,Suwannee,"{'type': 'Point', 'coordinates': [-82.97076400...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22
4,2020-05-24,105697,EAST BAY REHABILITATION CENTER,4470 E BAY DR,CLEARWATER,FL,33764,N,Pinellas,"{'type': 'Point', 'coordinates': [-82.740891, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21
5,2020-05-24,105803,HIALEAH NURSING AND REHABILITATION CENTER,190 W 28TH STREET,HIALEAH,FL,33010,N,Miami-Dade,"{'type': 'Point', 'coordinates': [-80.286094, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21
6,2020-05-24,105812,ROYAL CARE OF AVON PARK,1213 W STRATFORD RD,AVON PARK,FL,33825,N,Highlands,"{'type': 'Point', 'coordinates': [-81.51721, 2...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21
7,2020-05-24,105832,"GARDENS OF PORT ST LUCIE, THE",1699 SE LYNGATE DRIVE,PORT SAINT LUCIE,FL,34952,N,St. Lucie,"{'type': 'Point', 'coordinates': [-80.298828, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21
8,2020-05-24,105951,BRANDON HEALTH AND REHABILITATION CENTER,1465 OAKFIELD DR,BRANDON,FL,33511,N,Hillsborough,"{'type': 'Point', 'coordinates': [-82.307614, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21
9,2020-05-24,106040,WESTMINSTER ST AUGUSTINE,230 TOWERVIEW DRIVE,SAINT AUGUSTINE,FL,32092,N,St. Johns,"{'type': 'Point', 'coordinates': [-81.480016, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21


Rearange columns to match SQL Server columns order

In [4]:
df=df[['week_ending','WeekNo','provider_state',
       'residents_weekly_confirmed',
       'residents_total_confirmed',
       'residents_weekly_covid_19',
       'residents_total_covid_19',
       'staff_weekly_confirmed_covid',
       'staff_total_confirmed_covid',
       'staff_weekly_covid_19_deaths',
       'staff_total_covid_19_deaths']]
df.head(10)

,week_ending,WeekNo,provider_state,residents_weekly_confirmed,residents_total_confirmed,residents_weekly_covid_19,residents_total_covid_19,staff_weekly_confirmed_covid,staff_total_confirmed_covid,staff_weekly_covid_19_deaths,staff_total_covid_19_deaths
0,2020-05-24,21,AZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-05-24,21,CT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-05-24,21,FL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-05-31,22,FL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-05-24,21,FL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-05-24,21,FL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2020-05-24,21,FL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2020-05-24,21,FL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2020-05-24,21,FL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2020-05-24,21,FL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df['residents_weekly_confirmed']=pd.to_numeric(df['residents_weekly_confirmed'], downcast='integer')
df['residents_total_confirmed']=pd.to_numeric(df['residents_total_confirmed'], downcast='integer')
df['residents_weekly_covid_19']=pd.to_numeric(df['residents_weekly_covid_19'], downcast='integer')
df['residents_total_covid_19']=pd.to_numeric(df['residents_total_covid_19'], downcast='integer')
df['staff_weekly_confirmed_covid']=pd.to_numeric(df['staff_weekly_confirmed_covid'], downcast='integer')
df['staff_total_confirmed_covid']=pd.to_numeric(df['staff_total_confirmed_covid'], downcast='integer')
df['staff_weekly_covid_19_deaths']=pd.to_numeric(df['staff_weekly_covid_19_deaths'], downcast='integer')
df['staff_total_covid_19_deaths']=pd.to_numeric(df['staff_total_covid_19_deaths'], downcast='integer')
print (df.dtypes)

week_ending                     datetime64[ns]
WeekNo                                  UInt32
provider_state                          object
residents_weekly_confirmed             float64
residents_total_confirmed              float64
residents_weekly_covid_19              float64
residents_total_covid_19               float64
staff_weekly_confirmed_covid           float64
staff_total_confirmed_covid            float64
staff_weekly_covid_19_deaths           float64
staff_total_covid_19_deaths            float64
dtype: object


Calculate summarized rows

In [6]:
dfSum=df.groupby(by=['week_ending','WeekNo','provider_state'], as_index=False).agg({
            'residents_weekly_confirmed':'sum',
            'residents_total_confirmed':'sum',
            'residents_weekly_covid_19':'sum',
            'residents_total_covid_19':'sum',
            'staff_weekly_confirmed_covid':'sum',
            'staff_total_confirmed_covid':'sum',
            'staff_weekly_covid_19_deaths':'sum',
            'staff_total_covid_19_deaths':'sum'})
dfSum.head(10)

,week_ending,WeekNo,provider_state,residents_weekly_confirmed,residents_total_confirmed,residents_weekly_covid_19,residents_total_covid_19,staff_weekly_confirmed_covid,staff_total_confirmed_covid,staff_weekly_covid_19_deaths,staff_total_covid_19_deaths
0,2020-05-24,21,AK,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
1,2020-05-24,21,AL,1037.0,1037.0,231.0,231.0,691.0,691.0,3.0,3.0
2,2020-05-24,21,AR,336.0,336.0,44.0,44.0,254.0,254.0,0.0,0.0
3,2020-05-24,21,AZ,287.0,287.0,94.0,94.0,260.0,260.0,1.0,1.0
4,2020-05-24,21,CA,5556.0,5556.0,1219.0,1219.0,3613.0,3613.0,21.0,21.0
5,2020-05-24,21,CO,842.0,842.0,306.0,306.0,679.0,679.0,3.0,3.0
6,2020-05-24,21,CT,5731.0,5731.0,1558.0,1558.0,2202.0,2202.0,9.0,9.0
7,2020-05-24,21,DC,342.0,342.0,77.0,77.0,197.0,197.0,6.0,6.0
8,2020-05-24,21,DE,563.0,563.0,173.0,173.0,290.0,290.0,1.0,1.0
9,2020-05-24,21,FL,1890.0,1890.0,465.0,465.0,1272.0,1272.0,0.0,0.0


In [7]:
NursingHomesList=dfSum.values.tolist()

Connect to local SQL Server instance, dump & fill table

In [8]:
conn = pyodbc.connect('Driver={SQL Server};'
                    'Server=DESKTOP-BME09T7\MSSQLSERVER01;'
                    'Database=Covid19_NursingHomes;'
                    'Trusted_Connection=yes;', autocommit=True)
cursor = conn.cursor()

In [9]:
cursor.execute("TRUNCATE TABLE NursingHomes")

In [10]:
cursor.executemany("INSERT INTO NursingHomes(Week_Ending,WeekNo,"
                   "StateId,"
                   "Residents_Weekly_Confirmed_COVID,Residents_Total_Confirmed_COVID,"
                   "Residents_Weekly_Deaths_COVID,Residents_Total_Deaths_COVID,"
                   "Staff_Weekly_Confirmed_COVID,Staff_Total_Confirmed_COVID,Staff_Weekly_Deaths_COVID,Staff_Total_Deaths_COVID)"
                   " VALUES(?,?,?,?,?,?,?,?,?,?,?)", NursingHomesList)

In [11]:
cursor.rowcount

-1